In [57]:
import pandas as pd

#Recipes and Ingredients: Simple Cookbook
df_recipe_simple = pd.read_csv("data\\recipe.csv")
df_ingredient_simple = pd.read_csv("data\\ingredient.csv")

In [58]:
folder_path = "data\\atlas-query-cookbook"

df_ingredient = pd.read_csv(folder_path + r"\Ingredient", sep='\t')
df_ingredient['category'] = df_ingredient['category'].str.strip()
df_nutrition = pd.read_csv(folder_path + r"\Nutrition", sep='\t')
df_quantity = pd.read_csv(folder_path + r"\Quantity", sep='\t')
df_quantity['unit'] = df_quantity['unit'].str.strip()
df_recipe = pd.read_csv(folder_path + r"\Recipe", sep=',')

In [59]:
df_ingredient = (df_ingredient.drop(columns=['plural'])
                 .rename(columns = {"name": "ingredient_name"}))
new_order = ['ingredient_id', 'ingredient_name', 'category']
df_ingredient = df_ingredient[new_order]
df_ingredient

,ingredient_id,ingredient_name,category
0,1,1% lowfat cottage cheese,dairy
1,6,1% lowfat milk,dairy
2,10,10-inch flour tortilla,Mexican products
3,11,100% bran cereal,cereals
4,12,2% lowfat milk,dairy
...,...,...,...
3342,4641,cooked oatmeal,cereals
3343,4642,instant coffee granules,hot beverages
3344,4643,long grain enriched white rice,grains
3345,4644,frozen grapefruit juice concentrate,frozen fruit


In [60]:
df_quantity_ingredient = pd.merge(df_quantity, df_ingredient, on='ingredient_id')
df_quantity_ingredient["qty"] = (df_quantity_ingredient["min_qty"] + df_quantity_ingredient["max_qty"]) / 2
df_quantity_ingredient = (df_quantity_ingredient
                          .drop(columns = ["quantity_id", "max_qty", "min_qty", "preparation", "optional"]))

new_order = ['recipe_id', 'ingredient_id', 'ingredient_name', 'category', 'unit', 'qty']
df_quantity_ingredient = df_quantity_ingredient[new_order]
df_quantity_ingredient

,recipe_id,ingredient_id,ingredient_name,category,unit,qty
0,214,1613,graham cracker crumbs,cookies/crackers,cup(s),2.00
1,214,3334,sugar,baking products,cup(s),0.25
2,214,2222,margarine or butter,dairy,cup(s),0.50
3,214,2797,raspberry juice,fruit juices,cup(s),0.25
4,214,3567,unflavored gelatin,gelatin,teaspoon(s),3.00
...,...,...,...,...,...,...
5087,1119,289,whole boneless turkey,meat/poultry,pound(s),6.00
5088,1119,3261,Sonoma County chardonnay wine,alcoholic beverages,milliliter(s),750.00
5089,1119,638,chicken broth,soups,cup(s),1.00
5090,1119,1420,fresh rosemary,fresh herbs,teaspoon(s),3.00


In [61]:
def replacer(df, qty_type, multiplier, new_qty='gram(s)'):
    df.loc[df.unit == qty_type, 'qty'] *= multiplier
    df['unit'] = df['unit'].replace({qty_type: new_qty})
    return df

In [62]:
lst_qty = df_quantity.unit.unique()

for qty_type in lst_qty:
    ounce = qty_type.split(" ")[0]
    try:
        ounce_int = int(ounce)
    except:
        continue
    replacer(df_quantity_ingredient, qty_type, ounce_int * 29)
    
replacer(df_quantity_ingredient, 'pound(s)', 455)
replacer(df_quantity_ingredient, 'cup(s)', 240)
replacer(df_quantity_ingredient, 'ounce(s)', 29)
replacer(df_quantity_ingredient, 'fl. ounce(s)', 29)
replacer(df_quantity_ingredient, 'pint(s)', 12*29)
replacer(df_quantity_ingredient, 'drop(s)', 0.05, 'milliliter(s)')
replacer(df_quantity_ingredient, 'stalk(s)', 1, 'whole')
replacer(df_quantity_ingredient, 'head(s)', 1, 'whole')
replacer(df_quantity_ingredient, 'loaf', 1, 'whole')
replacer(df_quantity_ingredient, 'pinch(es)', 0.125, 'teaspoon(s)')
replacer(df_quantity_ingredient, 'dash(es)', 0.125, 'teaspoon(s)')
replacer(df_quantity_ingredient, 'packet(s)', 1, 'package(s)')
replacer(df_quantity_ingredient, 'envelope(s)', 1, 'package(s)')
replacer(df_quantity_ingredient, 'teaspoon(s)', 5)
replacer(df_quantity_ingredient, 'tablespoon(s)', 15)
replacer(df_quantity_ingredient, 'stick(s)', 1, 'piece(s)')
replacer(df_quantity_ingredient, 'slice(s)', 1, 'piece(s)')
replacer(df_quantity_ingredient, 'clove(s)', 1, 'piece(s)')
replacer(df_quantity_ingredient, 'scoop(s)', 1, 'piece(s)')
replacer(df_quantity_ingredient, 'bunch(es)', 1, 'piece(s)')
replacer(df_quantity_ingredient, 'sprig(s)', 1, 'piece(s)')

df_quantity_ingredient.loc[(df_quantity_ingredient.unit == 'quart(s)') & (df_quantity_ingredient.category.isin(['soups', 'beverages'])), 'qty'] *= 946
df_quantity_ingredient.loc[(df_quantity_ingredient.unit == 'quart(s)') & (df_quantity_ingredient.category.isin(['soups', 'beverages'])), 'unit'] = 'gram(s)'
replacer(df_quantity_ingredient, 'quart(s)', 1, 'piece(s)')

df_quantity_ingredient.where(df_quantity_ingredient.unit == 'gram(s)').dropna()   

,recipe_id,ingredient_id,ingredient_name,category,unit,qty
0,214.0,1613.0,graham cracker crumbs,cookies/crackers,gram(s),480.0
1,214.0,3334.0,sugar,baking products,gram(s),60.0
2,214.0,2222.0,margarine or butter,dairy,gram(s),120.0
3,214.0,2797.0,raspberry juice,fruit juices,gram(s),60.0
4,214.0,3567.0,unflavored gelatin,gelatin,gram(s),15.0
...,...,...,...,...,...,...
5086,1119.0,3618.0,vegetable oil,oils and fats,gram(s),30.0
5087,1119.0,289.0,whole boneless turkey,meat/poultry,gram(s),2730.0
5089,1119.0,638.0,chicken broth,soups,gram(s),240.0
5090,1119.0,1420.0,fresh rosemary,fresh herbs,gram(s),15.0


In [63]:
df_recipe_nutrition = pd.merge(df_recipe, df_nutrition, on='recipe_id')
df_recipe_nutrition = (df_recipe_nutrition.drop(columns=["yield_unit", "yield_unit", "prep_min", "cook_min", "stnd_min",
                                                         "source", "intro", "directions"]))

df_quantity_agg = (df_quantity[["recipe_id", "ingredient_id"]]
                    .groupby("recipe_id")["ingredient_id"]
                    .agg(list)
                    .reset_index()
                    .rename(columns = {"ingredient_id": "ingredients"}))
df_quantity_agg["ingredients"] = df_quantity_agg["ingredients"].apply(lambda x: sorted(x))

df_total_recipe = pd.merge(df_recipe_nutrition, df_quantity_agg, on = "recipe_id")

#Dividing the nutrition values by the servings
num_col = df_total_recipe.columns.difference(['recipe_id', 'title', 'subtitle', 'servings', 'ingredients'])
df_total_recipe[num_col] = df_total_recipe[num_col].div(df_total_recipe['servings'], axis=0)

dct_servings = {}
for i, v in enumerate(df_total_recipe.recipe_id):
    dct_servings[v] = df_total_recipe.servings[i]
    
df_total_recipe.drop(columns=['servings'], inplace=True)
df_total_recipe

,recipe_id,title,subtitle,protein,carbo,alcohol,total_fat,sat_fat,cholestrl,sodium,iron,vitamin_c,vitamin_a,fiber,pcnt_cal_carb,pcnt_cal_fat,pcnt_cal_prot,calories,ingredients
0,214,Raspberry Chiffon Pie,NaN,0.547000,4.129000,0.0000,1.153000,0.221000,0.139000,26.07800,0.081000,0.889000,58.620000,0.087000,5.680000,3.568000,0.7530,29.079000,"[924, 1414, 1613, 2196, 2222, 2797, 3334, 3334..."
1,215,Apricot Yogurt Parfaits,NaN,1.425000,5.937500,0.4825,0.270000,0.145000,0.870000,11.54250,0.142500,3.255000,684.560000,0.155000,16.845000,1.722500,4.0425,35.252500,"[71, 76, 1286, 2019, 2196, 3567]"
2,216,Fresh Apricot Bavarian,NaN,0.612500,3.360000,0.0000,0.137500,0.072500,0.432500,5.22375,0.046250,0.766250,190.137500,0.042500,9.806250,0.905000,1.7875,17.132500,"[1286, 2196, 3334, 3567, 3649]"
3,217,Fresh Peaches,with Banana Cream Whip,0.442500,4.542500,0.0000,0.052500,0.015000,0.000000,3.50250,0.047500,2.197500,119.522500,0.172500,22.245000,0.587500,2.1675,20.425000,"[792, 1402, 2019]"
4,218,Canned Cherry Crisp,NaN,0.230000,6.105000,0.0000,0.911667,0.576667,1.726667,8.37000,0.110000,0.026667,38.193333,0.175000,12.135000,4.076667,0.4550,33.538333,"[30, 34, 351, 374, 614, 770, 1684, 2948]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,1410,No-Bake Chocolate Cheesecake,NaN,0.541667,2.400833,0.0000,3.770000,2.040833,8.648333,14.42250,0.113333,0.049167,106.975000,0.046667,1.750833,6.186667,0.3950,45.703333,"[35, 249, 374, 374, 496, 782, 792, 921, 1755, ..."
839,1411,Mexican Cocoa Torte,NaN,0.324000,3.867000,0.0000,3.589000,1.467000,6.537000,23.87500,0.107000,0.030000,70.114000,0.009000,3.153000,6.583000,0.2640,49.067000,"[738, 1755, 1767, 2627, 3118, 3329, 3334]"
840,1426,-Pancakes-,NaN,1.572500,7.310000,0.0000,1.625000,0.445000,14.835000,61.18250,0.142500,0.107500,34.370000,0.020000,14.575000,7.287500,3.1375,50.157500,"[30, 126, 2327, 3021, 3334, 3595, 3618]"
841,1427,-Waffles-,NaN,0.821250,3.767500,0.0000,1.717500,0.356250,7.547500,21.73500,0.073750,0.063750,18.451250,0.012500,5.571250,5.713750,1.2150,33.812500,"[30, 125, 1696, 2327, 3021, 3334, 3618]"


In [65]:
for i, v in enumerate(df_quantity_ingredient.recipe_id):
    df_quantity_ingredient.loc[i, 'qty'] /= dct_servings[v]

In [66]:
def recipe_ingredients(df, recipe_id):
    return df[df["recipe_id"] == recipe_id]

recipe_ingredients(df_quantity_ingredient, 214)
recipe_ingredients(df_quantity_ingredient, 1020)

,recipe_id,ingredient_id,ingredient_name,category,unit,qty
2947,1020,756,cling peach halves in syrup,canned/bottled fruit,gram(s),154.666667
2948,1020,967,cucumber,fresh vegetables,gram(s),40.000000
2949,1020,1648,green onion,fresh vegetables,gram(s),20.000000
2950,1020,2442,olive oil,oils and fats,gram(s),5.000000
2951,1020,2196,plain lowfat yogurt,dairy,gram(s),38.666667
2952,1020,1573,garlic clove,fresh vegetables,piece(s),0.166667
2953,1020,1334,fresh ginger root,fresh herbs,gram(s),2.500000
2954,1020,2026,lemon pepper,spices and seasonings,gram(s),0.416667
2955,1020,1029,dill,spices and seasonings,gram(s),0.416667
2956,1020,804,cooked chicken,meat/poultry,gram(s),120.000000
